# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [49]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [50]:
weather_df = pd.read_csv("output.csv")
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,0,albany,42.6001,-73.9662,60.03,86,39,3.13,1970-01-01,US
1,1,awbari,26.5921,12.7805,94.57,13,0,10.04,1970-01-01,LY
2,2,avarua,-21.2078,-159.7750,77.05,73,100,19.57,1970-01-01,CK
3,3,rikitea,-23.1203,-134.9692,70.95,71,95,22.84,1970-01-01,PF
4,4,atuona,-9.8000,-139.0333,77.32,64,100,14.05,1970-01-01,PF


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [51]:
gmaps.configure(api_key=g_key)
# Create initial figure
fig = gmaps.figure()

# Create Heatmap layer
locations = weather_df[["Lat", "Lng"]]
heatmap_layer = gmaps.heatmap_layer(locations, weights=weather_df["Humidity"])
fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [52]:
#create good weather conditions for holiday
idx = np.where((weather_df['Wind Speed']< 10)&
               (weather_df['Max Temp']>70)&
               (weather_df['Max Temp']<80)&
               (weather_df['Cloudiness']==0))


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [53]:
#store in hotel_df dataframe
hotel_df = weather_df.loc[idx]
#create new column
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country,Hotel Name
51,51,namibe,-15.1961,12.1522,77.85,41,0,3.87,1970-01-01,AO,
66,66,pochutla,15.7432,-96.4661,78.82,85,0,4.34,1970-01-01,MX,
228,228,okhotsk,59.3833,143.3000,70.81,59,0,6.02,1970-01-01,RU,
250,250,inhambane,-23.8650,35.3833,72.19,55,0,2.10,1970-01-01,MZ,
290,290,ampanihy,-24.7000,44.7500,74.79,35,0,6.24,1970-01-01,MG,
426,426,namtsy,62.7161,129.6658,75.31,24,0,4.68,1970-01-01,RU,
518,518,aykhal,66.0000,111.5000,78.87,26,0,5.19,1970-01-01,RU,


In [54]:
#copy lat and longitude to a list zipped together
lat =hotel_df['Lat'].to_list()
lng = hotel_df['Lng'].to_list()
lat_lng = list(zip(lat, lng))

#create variables for params
target_coordinates = lat_lng
target_radius = 5000
target_type = "hotel"

#set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

#requesting the hotels and saving the first hotel in hotel name
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    hotel_address = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/Result ... skipping")
    
hotel_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country,Hotel Name
51,51,namibe,-15.1961,12.1522,77.85,41,0,3.87,1970-01-01,AO,Namibe
66,66,pochutla,15.7432,-96.4661,78.82,85,0,4.34,1970-01-01,MX,Pochutla District
228,228,okhotsk,59.3833,143.3000,70.81,59,0,6.02,1970-01-01,RU,Okhotsk
250,250,inhambane,-23.8650,35.3833,72.19,55,0,2.10,1970-01-01,MZ,Maxixe
290,290,ampanihy,-24.7000,44.7500,74.79,35,0,6.24,1970-01-01,MG,Ampanihy
426,426,namtsy,62.7161,129.6658,75.31,24,0,4.68,1970-01-01,RU,Namtsy
518,518,aykhal,66.0000,111.5000,78.87,26,0,5.19,1970-01-01,RU,Aikhal


In [55]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

marker_locations = hotel_df[['Lat', 'Lng']]
hotel_name = hotel_df["Hotel Name"].tolist()
max_temp = hotel_df["Max Temp"]

#create two layers
heatmap_layer = gmaps.heatmap_layer(marker_locations, weights=hotel_df["Humidity"])
markers = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))